## 2. Haar 방식 얼굴, 눈 식별<br>
### (Face, eye Detection)<br>
object detection  

### Haar-cascade detection 이란?

1. Object Detection (기본적인 방식)  
2. Deep Neural Networks (DNN module)  

    당연히 DNN 모델이 정확도를 갖고 있다.  
    나중에 졸음방지 등을 할 때에, 1과 2를 융합해서 사용하면 퍼포먼스가 훨씬 좋을 수 있다.

1_1. Cascade Classifier  
- Haar cascade object detection  
: CNN 와 비슷한 컨셉을 갖고 있으나 다른 방법으로 빠르게 인식하는 방식  
: (장점) 굉장히 빠르게 오브젝트를 인식 해준다. 심플, 가볍다  
: (단점) 정확도가 CNN보다 떨어진다. : 예외상황에 인식률이 현저하게 떨어진다. - 그래서 둘다 조합해서 사용한다.  
: 미리 정해준 방식에 의해서 인식을 하는 것 (DNN과의 차이는 Train의 차이)  


## 2-2. Haar 방식을 이용한 Face Dectection

In [1]:
import cv2
import numpy as np

def detectAndDisplay(frame):
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.equalizeHist(frame_gray)
    #-- Detect faces
    faces = face_cascade.detectMultiScale(frame_gray)
    for (x,y,w,h) in faces:
        center = (x + w//2, y + h//2)   # 얼굴이 베이스면 그 중간
        frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 4)
        faceROI = frame_gray[y:y+h, x:x+w]  # 금방 선택한 얼굴만 끄집어 낸다
        #-- In each face, detect eyes
        eyes = eyes_cascade.detectMultiScale(faceROI)
        for (x2,y2,w2,h2) in eyes:
            eye_center = (x + x2 + w2//2, y + y2 + h2//2)
            radius = int(round((w2 + h2)*0.25))
            frame = cv2.circle(frame, eye_center, radius, (255, 0, 0), 4)
        cv2.imshow('Capture - Face detection', frame)

print('OpenCV version :', cv2.__version__)

OpenCV version : 4.3.0


In [2]:
img = cv2.imread('./source/opencv_dnn_202005/image/marathon_01.jpg')
print('w : {} pixels'.format(img.shape[1]))
print('h : {} pixels'.format(img.shape[0]))
print('ch : {} '.format(img.shape[2]))

(height, width) = img.shape[:2]

cv2.imshow('Original Image', img)

face_cascade_name = './source/ai_cv/haarcascades/haarcascade_frontalface_alt.xml'
eyes_cascade_name = './source/ai_cv/haarcascades/haarcascade_eye_tree_eyeglasses.xml'

face_cascade = cv2.CascadeClassifier()
eyes_cascade = cv2.CascadeClassifier()

#-- 1. Load the cascades
if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print('--(!)Error loading face cascade')
    exit(0)
if not eyes_cascade.load(cv2.samples.findFile(eyes_cascade_name)):
    print('--(!)Error loading eyes cascade')
    exit(0)
    
detectAndDisplay(img)

cv2.waitKey(0)
cv2.destroyAllWindows()

w : 1024 pixels
h : 799 pixels
ch : 3 


## 2-3. Haar 방식 Face Detection GUI 프로그램<br>
pip install Pillow

In [4]:
import cv2
import numpy as np
from tkinter import *
from PIL import Image
from PIL import ImageTk
from tkinter import filedialog

face_cascade_name = './source/ai_cv/haarcascades/haarcascade_frontalface_alt.xml'
eyes_cascade_name = './source/ai_cv/haarcascades/haarcascade_eye_tree_eyeglasses.xml'
file_name = './source/opencv_dnn_202005/image/marathon_01.jpg'
title_name = 'Haar cascade object detection'
frame_width = 500     # size를 크게하면, 인식률이 좋아진다.

# 함수1
def selectFile():
    file_name = filedialog.askopenfilename(initialdir = './source/opencv_dnn_202005/image', title = 'Select file', filetypes = (('jpeg files','*.jpg'),('all files','*.*')))
    print('File name : ', file_name)
    read_image = cv2.imread(file_name)
    (height, width) = read_image.shape[:2]
    frameSize = int(sizeSpin.get())
    ratio = frameSize / width
    dimension = (frameSize, int(height * ratio))
    read_image = cv2.resize(read_image, dimension, interpolation = cv2.INTER_AREA)
    image = cv2.cvtColor(read_image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    imgtk = ImageTk.PhotoImage(image=image)
    detectAndDisplay(read_image)
    
# 함수2
def detectAndDisplay(frame):
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.equalizeHist(frame_gray)
    #-- Detect faces
    faces = face_cascade.detectMultiScale(frame_gray)
    for (x,y,w,h) in faces:
        center = (x + w//2, y + h//2)   # 얼굴이 베이스면 그 중간
        frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 4)
        faceROI = frame_gray[y:y+h, x:x+w]  # 금방 선택한 얼굴만 끄집어 낸다
        #-- In each face, detect eyes
        eyes = eyes_cascade.detectMultiScale(faceROI)
        for (x2,y2,w2,h2) in eyes:
            eye_center = (x + x2 + w2//2, y + y2 + h2//2)
            radius = int(round((w2 + h2)*0.25))
            frame = cv2.circle(frame, eye_center, radius, (255, 0, 0), 4)
    # cv2.imshow('Capture - Face detection', frame)
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    imgtk = ImageTk.PhotoImage(image=image)
    detection.config(image=imgtk)
    detection.image = imgtk

# main
main = Tk()
main.title(title_name)
main.geometry()
        
# 이미지 불러오기
read_image = cv2.imread('./source/opencv_dnn_202005/image/marathon_01.jpg')
(height, width) = read_image.shape[:2]

# 이미지 리사이즈
ratio = frame_width / width
dimension = (frame_width, int(height * ratio))
read_image = cv2.resize(read_image, dimension, interpolation = cv2.INTER_AREA)

# 이미지 RGB화 (안하면 색이 이상하게 나온다)
image = cv2.cvtColor(read_image, cv2.COLOR_BGR2RGB)
image = Image.fromarray(image)
imgtk = ImageTk.PhotoImage(image=image)  # 바로 보여주는 것이 아니고, tkinter에 pillow 보여주기 위해

face_cascade_name = './source/ai_cv/haarcascades/haarcascade_frontalface_alt.xml'
eyes_cascade_name = './source/ai_cv/haarcascades/haarcascade_eye_tree_eyeglasses.xml'

face_cascade = cv2.CascadeClassifier()
eyes_cascade = cv2.CascadeClassifier()

#-- 1. Load the cascades
if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print('--(!)Error loading face cascade')
    exit(0)
if not eyes_cascade.load(cv2.samples.findFile(eyes_cascade_name)):
    print('--(!)Error loading eyes cascade')
    exit(0)

label=Label(main, text=title_name)
label.config(font=("Courier", 18))
label.grid(row=0,column=0,columnspan=4)
sizeLabel=Label(main, text='Frame Width : ')                
sizeLabel.grid(row=1,column=0)
sizeVal  = IntVar(value=frame_width)
sizeSpin = Spinbox(main, textvariable=sizeVal,from_=0, to=2000, increment=100, justify=RIGHT)
sizeSpin.grid(row=1, column=1)
Button(main,text="File Select", height=2,command=lambda:selectFile()).grid(row=1, column=2, columnspan=2, sticky=(W, E))
detection=Label(main, image=imgtk)
detection.grid(row=2,column=0,columnspan=4)
detectAndDisplay(read_image)

main.mainloop()